In [1]:
# !pip install geemap

# EO4BEE

**This webapp provide a tool to visualize various earth observation data to support the decision-making process of beekeeping in Europe**

1. Select the earth observation variable that you want to explore
2. Select the time period that you're interested in.
3. Drag the time slider to visualize the image at a specific time which you want to visualize. 
4. Click on the tool icon and then the 'i' icon to inspect value at a specific location, or the plot icon to plot the change of the chosen variable over time at a specific location.
5. If you want to visualize the change smoothly as a gif, use the create timelapse tool to create and download customized timelapse gif.

In [2]:
# import all the required libraries
import os
import ee
import geemap
import ipywidgets as widgets
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
# initialize the needed widgets

# default style
style = {'description_width': 'initial'}

# layer selector dropdown box
layer = widgets.Dropdown(
    description='Select Layer:',
    options=['Annual Average Temperature','Monthly Average Temperature','Annual Total Precipitation',
             'Monthly Total Precipitation','NDVI','Landcover','Soil Moisture', 'PM2.5','Sulphur Dioxide'],
    value='Annual Average Temperature',
    style=style
)

# function to get the available time span for each dataset accordingly
def getAvailableYears(layer):
    if layer == 'Annual Average Temperature' or layer == 'Monthly Average Temperature' or layer == 'Annual Total Precipitation' or layer == 'Monthly Total Precipitation':
        return ['1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', 
                '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', 
                '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', 
                '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', 
                '2015', '2016', '2017', '2018', '2019', '2020']
    if layer == 'NDVI':
        return ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', 
                '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', 
                '2017', '2018', '2019', '2020', '2021']
    if layer == 'Landcover':
        return ['1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', 
                '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', 
                '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', 
                '2013', '2014', '2015', '2016', '2017', '2018']
    if layer == 'Soil Moisture':
        return ['1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', 
                '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', 
                '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', 
                '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', 
                '2018', '2019', '2020', '2021']
    if layer == 'PM2.5':
        return ['2016', '2017', '2018', '2019', '2020', '2021']
    if layer == 'Sulphur Dioxide':
        return ['2018', '2019', '2020', '2021']
    
# function to generate the year selector according to the selected layer
year_selector = widgets.Output()

# initialize the start_year and end_year as global variable so that their value can be obtained during the visualization
start_year = widgets.Dropdown()
end_year = widgets.Dropdown()

# function to show the year selector with the right range for different selected layer
def getYearSelector(layer):
    year_selector.clear_output()
    timespan = getAvailableYears(layer)
    # dropdown box to select the start year
    global start_year
    start_year = widgets.Dropdown(
        description='Start year:',
        options= timespan,
        value=timespan[0],
        style=style
    )
    # dropdown box to select the end year
    global end_year
    end_year = widgets.Dropdown(
        description='End year:',
        options= timespan,
        value=timespan[len(timespan)-1],
        style=style
    )
    with year_selector:
        display(widgets.HBox([start_year, end_year]))
        

# initialize the year selector
getYearSelector(layer.value)

# callback function for the change of the layer selector's value
def on_layer_change(change): 
    year_selector.clear_output()
    layer = change['new']
    getYearSelector(layer)
#     print(start_year.value,end_year.value)
    
# when the layer is changed, change the range of the year selector accordingly
layer.observe(on_layer_change, names='value')

# button to visualize the selected layer
show_layer = widgets.Button(
    description='Visualize',
    button_style='primary',
    tooltip='Click to visualize the selected layer',
    style=style
)

# adjust alignment of the horizontal layout box
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    justify_content = 'space-between',
                    border='none',
                    width='100%',
                    height = '40px')

# horizontal layout box
hbox1 = widgets.HBox(children = [layer, year_selector, show_layer], layout = box_layout)
hbox1

In [4]:
# Map.remove_control(Map.slider_ctrl)
# Map.add_control(Map.slider_ctrl)
# Map.slider_ctrl.widget.children[4].click()

In [5]:
Map = geemap.Map()
Map.set_center(14, 52, 4)
Map

Map(center=[52, 14], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [6]:
# function converting fahrenheit to celsius
def toCelsius(image):
    celsius = image.subtract(273.15) 
    return celsius
def getLabels(start_date,end_date,description):
    labels = []
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    current = start
    while current <= end:
        labels.append(datetime.strftime(current, '%Y'))
        current += relativedelta(years=1)
    for i,label in enumerate(labels):
        labels[i]=label+description
    return labels

In [7]:
# visualize the selected layer in the selected time span
def showLayer():
    # set the time span according to the selected year value
    start_date = start_year.value + '-01-01'
    end_date = end_year.value + '-12-31'
    # visualize the dataset according to the selected layer
    if layer.value == 'Annual Average Temperature':
        era5_complete = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('mean_2m_air_temperature').filterDate(start_date,end_date)
        distinctYear = era5_complete.distinct('year')
        filter = ee.Filter.equals(leftField='year', rightField= 'year')
        join = ee.Join.saveAll('sameYear')
        joinCol = ee.ImageCollection(
          join.apply(distinctYear, era5_complete, filter)
        )

        def func_lgr(img):
          yearCol = ee.ImageCollection.fromImages(img.get('sameYear'))
          return yearCol.mean().set('Year', img.get('year'))

        annualMeanCol = joinCol.map(func_lgr)
        annualMeanCol_celsius = annualMeanCol.map(toCelsius)
        image = annualMeanCol_celsius.toBands()
        vis_params = {
            'min': -23.15,
            'max': 46.85,
            'opacity': 0.7,
            'palette': [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
            ]
        }
    #     Map.addLayer(image, {}, "Time series", False)
        Map.add_time_slider(image, vis_params, '','Time series of annual average air temperature(℃) at 2m', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Monthly Average Temperature':
        collection = ee.ImageCollection('ECMWF/ERA5/MONTHLY').select('mean_2m_air_temperature').filterDate(start_date,end_date)
        vis_params = {
            'min': -23.15,
            'max': 46.85,
            'opacity': 0.7,
            'palette': [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
            ]
        }
        era5_celsius = collection.map(toCelsius)
        image = era5_celsius.toBands()
        Map.add_time_slider(image, vis_params, '','Time series of monthly average air temperature(℃) at 2m', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Annual Total Precipitation':
        era5_complete = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('total_precipitation').filterDate(start_date,end_date)
        distinctYear = era5_complete.distinct('year')
        filter = ee.Filter.equals(leftField='year', rightField= 'year')
        join = ee.Join.saveAll('sameYear')
        joinCol = ee.ImageCollection(
          join.apply(distinctYear, era5_complete, filter)
        )

        def func_lgr(img):
          yearCol = ee.ImageCollection.fromImages(img.get('sameYear'))
          return yearCol.sum().set('Year', img.get('year'))

        annualSumCol = joinCol.map(func_lgr)
        image = annualSumCol.toBands()
        vis_params = {
              'min': 0,
              'max': 4.8,
              'opacity': 0.7,
              'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
            }
    #     Map.addLayer(image, {}, "Time series", False)
        Map.add_time_slider(image, vis_params, '','Time series of annual average air temperature(℃) at 2m', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Monthly Total Precipitation':
        collection  = ee.ImageCollection('ECMWF/ERA5/MONTHLY').select('total_precipitation').filterDate(start_date,end_date)
        vis_params = {
              'min': 0,
              'max': 0.4,
              'opacity': 0.7,
              'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
            }
        image = collection.toBands()
        Map.add_time_slider(image, vis_params, '','Time series of monthly total precipitation(m)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'NDVI':
        collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_8DAY_NDVI').select('NDVI').filterDate(start_date,end_date)
        vis_params = {
          'min': 0.0,
          'max': 1.0,
          'opacity': 0.7,
          'palette': [
            'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
            '012E01', '011D01', '011301'
          ],
        }
#         Map.addLayer(collection, vis_params, 'NDVI Reference')
        image = collection.toBands()
        Map.add_time_slider(collection, vis_params, '','Time series of normalized difference vegetation index(NDVI)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------   
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Landcover':
        collection = ee.ImageCollection('COPERNICUS/CORINE/V20/100m').select('landcover').filterDate(start_date,end_date)
        vis_params = {}
        image = collection.toBands()
        Map.add_time_slider(collection, vis_params, '','Time series of CORINE landcover', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Soil Moisture':
        collection  = ee.ImageCollection('NASA/FLDAS/NOAH01/C/GL/M/V001').select('SoilMoi00_10cm_tavg').filterDate(start_date,end_date)
        vis_params = {
          'min': 0.0,
          'max': 1.0,
          'opacity': 0.7,
          'palette': ['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529']
        }
        image = collection.toBands()
        Map.add_time_slider(image, vis_params, '','Time series of soil moisture (0 - 10 cm underground) in m^3 m^-3', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'PM2.5':
        collection  = ee.ImageCollection('ECMWF/CAMS/NRT').select('particulate_matter_d_less_than_25_um_surface').filterDate(start_date,end_date)
        vis_params = {
          'min': 0.0,
          'max': 0.000076,
          'opacity': 0.7,
          'palette': ["5E4FA2","3288BD","66C2A5","ABE0A4","E6F598",
                      "FFFFBF","FEE08B","FDAE61","F46D43","D53E4F","9E0142"]
        }
        collection_yearly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='year', reducer='mean',drop_empty=False)
        labels = getLabels(start_date,end_date,' (particulate_matter_d_less_than_25_um_surface)')
        image = collection_yearly_mean.toBands()
        Map.add_time_slider(image, vis_params, labels=labels, layer_name='Time series of Particulate matter d < 2.5 um(kg m^-3)', time_interval=2)
#----------------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------------   
    if layer.value == 'Sulphur Dioxide':
        collection  = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2').select('SO2_column_number_density').filterDate(start_date,end_date)
        
#         distinctDay = collection.distinct('TIME_REFERENCE_DAYS_SINCE_1950')
#         filter = ee.Filter.equals(leftField='TIME_REFERENCE_DAYS_SINCE_1950', rightField= 'TIME_REFERENCE_DAYS_SINCE_1950')
#         join = ee.Join.saveAll('sameDay')
#         joinCol = ee.ImageCollection(
#           join.apply(distinctDay, collection, filter)
#         )

#         def func_lgr(img):
#           dayCol = ee.ImageCollection.fromImages(img.get('sameDay'))
#           return dayCol.mean().set('Day', img.get('TIME_REFERENCE_DAYS_SINCE_1950'))

#         dailyMeanCol = joinCol.map(func_lgr)
        collection_yearly_mean = geemap.create_timeseries(collection, start_date, end_date, frequency='year', reducer='mean',drop_empty=False)
        vis_params = {
          'min': 0.0,
          'max': 0.0005,
          'opacity': 0.7,
          'palette':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
        }
        
        labels = getLabels(start_date,end_date,' (SO2_column_number_density)')
        image = collection_yearly_mean.toBands()
        Map.add_time_slider(image, vis_params,labels=labels, layer_name='Time series of SO2 vertical column density at ground level(mol/m^2)', time_interval=2)
      

In [8]:
def getLayerName(layer_now):
    if layer_now == 'Monthly Average Temperature':
        return 'Time series of monthly average air temperature(℃) at 2m'
    elif layer_now == "Annual Average Temperature":
        return 'Time series of annual average air temperature(℃) at 2m'
    else:
        return layer_now

def vis_clicked(b):
#     remove current layers and slider control
    if hasattr(Map,'slider_ctrl'):
        Map.slider_ctrl.widget.children[4].click()
    showLayer()
    
show_layer.on_click(vis_clicked)

<br>
<strong> &nbsp; &nbsp;Create Timelapse:</strong>

In [9]:
ImgCollList = {
    'Temperature':'ECMWF/ERA5/MONTHLY',
    'Precipitation':'ECMWF/ERA5/MONTHLY',
    'NDVI':'LANDSAT/LE07/C01/T1_8DAY_NDVI',
    'Landcover':'COPERNICUS/CORINE/V20/100m',
    'Soil Moisture':'NASA/FLDAS/NOAH01/C/GL/M/V001',
    'PM2.5':'ECMWF/CAMS/NRT',
    'Sulphur Dioxide':'COPERNICUS/S5P/NRTI/L3_SO2',
}
BandList = {
    'Temperature':'mean_2m_air_temperature',
    'Precipitation':'total_precipitation',
    'NDVI':'NDVI',
    'Landcover':'landcover',
    'Soil Moisture':'SoilMoi00_10cm_tavg',
    'PM2.5':'particulate_matter_d_less_than_25_um_surface',
    'Sulphur Dioxide':'SO2_column_number_density',
}
PaletteList = {
    'Temperature':[
            "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
            "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
            "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
        ],
    'Precipitation':['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081'],
    'NDVI':[
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
        ],
    'Landcover':[],
    'Soil Moisture':['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529'],
    'PM2.5':["5E4FA2","3288BD","66C2A5","ABE0A4","E6F598","FFFFBF","FEE08B","FDAE61","F46D43","D53E4F","9E0142"],
    'Sulphur Dioxide':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
VisParamList = {
    'Temperature':{
        'bands':['mean_2m_air_temperature'],
        'min': 250,
        'max': 320,
        'palette': [
            "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
            "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
            "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
        ]
    },
    'Precipitation':{
        'bands':['total_precipitation'],
        'min': 0,
        'max': 0.4,
        'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
    },
    'NDVI':{
        'bands':['NDVI'],
        'min': 0.0,
        'max': 1.0,
        'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
        ],
    },
    'Landcover':{'bands':['landcover']},
    'Soil Moisture':{
        'bands':['SoilMoi00_10cm_tavg'],
        'min': 0.0,
        'max': 1.0,
        'palette': ['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529']
    },
    'PM2.5':{
        'bands':['particulate_matter_d_less_than_25_um_surface'],
        'min': 0.0,
        'max': 0.000076,
        'palette': ["5E4FA2","3288BD","66C2A5","ABE0A4","E6F598",
                  "FFFFBF","FEE08B","FDAE61","F46D43","D53E4F","9E0142"]
    },
    'Sulphur Dioxide':{
        'bands':['SO2_column_number_density'],
        'min': 0.0,
        'max': 0.0005,
        'palette':['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
    },
}
ReducerList = {
    'Temperature':'mean',
    'Precipitation':'sum',
    'NDVI':'median',
    'Landcover':'median',
    'Soil Moisture':'mean',
    'PM2.5':'mean',
    'Sulphur Dioxide':'mean',  
}
FrequencyList = {
    'Temperature':['year','month'],
    'Precipitation':['year','month'],
    'NDVI':['year','month'],
    'Landcover':['year'],
    'Soil Moisture':['year','month','day'],
    'PM2.5':['year','month','day'],
    'Sulphur Dioxide':['year','month','day'],  
}
# layer selector dropdown box
dataset = widgets.Dropdown(
    description='Select Dataset:',
    options=['Temperature','Precipitation','NDVI','Landcover','Soil Moisture', 'PM2.5','Sulphur Dioxide'],
    value='Temperature',
    style=style
)
dataset

Dropdown(description='Select Dataset:', options=('Temperature', 'Precipitation', 'NDVI', 'Landcover', 'Soil Mo…

In [10]:
# Map to show the timelapse
Map1 = geemap.Map()
Map1.set_center(14, 52, 4)
Map1

Map(center=[52, 14], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [11]:
widgets_layout = widgets.Layout(width='30%')
title = widgets.Text(
    value='Timelapse',
    placeholder='Enter title for the timelapse',
    description='Title:',
    disabled=False,
    style=style,
    layout = widgets_layout
)
start_date = widgets.DatePicker(
    description='Start date:',
    disabled=False,
    style=style,
    layout = widgets_layout
)
end_date = widgets.DatePicker(
    description='End date:',
    disabled=False,
    style=style,
    layout = widgets_layout
)
speed = widgets.IntSlider(
    value=5,
    min=1,
    max=30,
    step=1,
    description='Frames per second:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style,
    layout = widgets_layout
)
add_progress_bar = widgets.Checkbox(
    value=True,
    description='Add progress bar',
    disabled=False,
    indent=False,
    style=style,
    layout = widgets_layout
)
progress_bar_color = widgets.ColorPicker(
    concise=False,
    description='Progress bar color:',
    value='#0000ff',
    disabled=False,
    style=style,
    layout = widgets_layout
)
font_size = widgets.IntSlider(
    value=25,
    min=10,
    max=50,
    step=1,
    description='Font size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style,
    layout = widgets_layout
)
font_color = widgets.ColorPicker(
    concise=False,
    description='Font color:',
    value='#ffffff',
    disabled=False,
    style=style,
    layout = widgets_layout
)
frequency = widgets.Dropdown(
    options=FrequencyList[dataset.value],
    value='year',
    description='Temporal frequency:',
    disabled=False,
    style=style,
    layout = widgets_layout
)
submit = widgets.Button(
    description='Create Timelapse',
    button_style='primary',
    tooltip='Click to create timelapse',
    style=style,
)


hbox2 = widgets.HBox(children=[title,start_date,end_date],layout = box_layout)
hbox3 = widgets.HBox([speed,add_progress_bar,progress_bar_color],layout = box_layout)
hbox4 = widgets.HBox([font_size,font_color,frequency],layout = box_layout)
hbox5 = widgets.HBox([submit],layout = box_layout)
vbox = widgets.VBox([hbox2,hbox3,hbox4,hbox5])
vbox

In [12]:
output = widgets.Output()
output

Output()

In [13]:
def submit_clicked(b):
    with output:
        output.clear_output()
        if start_date.value >= end_date.value:
            print('The end date must be later than the start date.')
            return
        print('Computing...')
        
        try:
            feature = Map1.draw_last_feature
            if feature is None:
                print('Please draw a feature on the map to specify the region of interest')
                return
            roi = feature.geometry()
#             out_gif = geemap.temp_file_path(".gif")
            out_gif = os.path.join(os.path.expanduser('~'), 'Downloads',title.value+'.gif')
            VisParams = VisParamList[dataset.value]
            if dataset.value == 'Precipitation' and frequency.value == 'year':
                VisParams['max'] = 4.8
            geemap.create_timelapse(
                collection = ImgCollList[dataset.value],
                start_date=start_date.value.strftime("%Y-%m-%d"),
                end_date=end_date.value.strftime("%Y-%m-%d"),
                region=roi,
                frequency=frequency.value,
                reducer=ReducerList[dataset.value],
                date_format='YYYY-MM-dd',
                out_gif=out_gif,
                bands=BandList[dataset.value],
                palette=PaletteList[dataset.value],
                vis_params=VisParams,
                dimensions=768,
                frames_per_second=speed.value,
                crs="EPSG:3857",
                title=title.value,
                title_xy=("2%", "90%"),
                add_text=True,
                text_xy=("2%", "2%"),
                text_sequence=None,
                font_size=font_size.value,
                font_color=font_color.value,
                add_progress_bar=add_progress_bar.value,
                progress_bar_color=progress_bar_color.value,
                progress_bar_height=5,
                loop=0,
            )
        except:
            empty_text.error(
                "An error occurred while computing the timelapse. You probably requested too much data. Try reducing the ROI or timespan."
            )    
            
submit.on_click(submit_clicked)